In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [3]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [4]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [5]:
keras_reg.fit(X_train, y_train, epochs=100,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 0s 28us/sample - loss: 1.0965 - val_loss: 1.3712
Epoch 2/100
11610/11610 [==============================] - 0s 21us/sample - loss: 0.5741 - val_loss: 1.8186
Epoch 3/100
11610/11610 [==============================] - 0s 20us/sample - loss: 0.5187 - val_loss: 0.8305
Epoch 4/100
11610/11610 [==============================] - 0s 22us/sample - loss: 0.4841 - val_loss: 0.4988
Epoch 5/100
11610/11610 [==============================] - 0s 22us/sample - loss: 0.4588 - val_loss: 0.4278
Epoch 6/100
11610/11610 [==============================] - 0s 21us/sample - loss: 0.4447 - val_loss: 0.4293
Epoch 7/100
11610/11610 [==============================] - 0s 20us/sample - loss: 0.4334 - val_loss: 0.4007
Epoch 8/100
11610/11610 [==============================] - 0s 21us/sample - loss: 0.4249 - val_loss: 0.3922
Epoch 9/100
11610/11610 [==============================] - 0s 21us/sample - loss: 0.417

In [6]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] learning_rate=0.005237378394707071, n_hidden=2, n_neurons=31 ....


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 0s 36us/sample - loss: 1.0129 - val_loss: 2.8879
Epoch 2/100
7740/7740 [==============================] - 0s 25us/sample - loss: 0.6066 - val_loss: 9.1680
Epoch 3/100
7740/7740 [==============================] - 0s 24us/sample - loss: 0.5786 - val_loss: 0.4399
Epoch 4/100
7740/7740 [==============================] - 0s 24us/sample - loss: 0.4475 - val_loss: 0.4237
Epoch 5/100
7740/7740 [==============================] - 0s 26us/sample - loss: 0.4216 - val_loss: 0.3979
Epoch 6/100
7740/7740 [==============================] - 0s 24us/sample - loss: 0.4074 - val_loss: 0.4230
Epoch 7/100
7740/7740 [==============================] - 0s 25us/sample - loss: 0.3987 - val_loss: 0.3726
Epoch 8/100
7740/7740 [==============================] - 0s 25us/sample - loss: 0.3916 - val_loss: 0.3869
Epoch 9/100
7740/7740 [==============================] - 0s 24us/sample - loss: 0.3846 - val_loss: 0.410

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.9s remaining:    0.0s


7740/7740 [==============================] - 0s 37us/sample - loss: 0.9764 - val_loss: 0.8133
Epoch 2/100
7740/7740 [==============================] - 0s 25us/sample - loss: 0.5693 - val_loss: 0.8195
Epoch 3/100
7740/7740 [==============================] - 0s 24us/sample - loss: 0.4857 - val_loss: 1.4969
Epoch 4/100
7740/7740 [==============================] - 0s 24us/sample - loss: 0.4440 - val_loss: 1.2916
Epoch 5/100
7740/7740 [==============================] - 0s 25us/sample - loss: 0.4231 - val_loss: 0.7994
Epoch 6/100
7740/7740 [==============================] - 0s 26us/sample - loss: 0.4089 - val_loss: 0.6993
Epoch 7/100
7740/7740 [==============================] - 0s 26us/sample - loss: 0.4001 - val_loss: 0.4929
Epoch 8/100
7740/7740 [==============================] - 0s 25us/sample - loss: 0.3932 - val_loss: 0.4130
Epoch 9/100
7740/7740 [==============================] - 0s 25us/sample - loss: 0.3866 - val_loss: 0.3655
Epoch 10/100
7740/7740 [==============================] - 

...

7740/7740 [==============================] - 0s 28us/sample - loss: 0.3385 - val_loss: 0.3398
Epoch 49/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.3381 - val_loss: 0.3398
Epoch 50/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.3371 - val_loss: 0.3357
Epoch 51/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.3362 - val_loss: 0.3430
Epoch 52/100
7740/7740 [==============================] - 0s 27us/sample - loss: 0.3359 - val_loss: 0.3357
Epoch 53/100
7740/7740 [==============================] - 0s 27us/sample - loss: 0.3353 - val_loss: 0.3361
Epoch 54/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.3349 - val_loss: 0.3383
Epoch 55/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.3336 - val_loss: 0.3447
Epoch 56/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.3334 - val_loss: 0.3371
Epoch 57/100
7740/7740 [==========================

7740/7740 [==============================] - 0s 28us/sample - loss: 0.3900 - val_loss: 0.3611
Epoch 29/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.3878 - val_loss: 0.3778
Epoch 30/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.3850 - val_loss: 0.3928
Epoch 31/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.3837 - val_loss: 0.3840
Epoch 32/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.3813 - val_loss: 0.3546
Epoch 33/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.3798 - val_loss: 0.4013
Epoch 34/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.3782 - val_loss: 0.3796
Epoch 35/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.3769 - val_loss: 0.3562
Epoch 36/100
7740/7740 [==============================] - 0s 28us/sample - loss: 0.3751 - val_loss: 0.3941
Epoch 37/100
7740/7740 [==========================

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  4.4min finished
/Users/prabesh/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 0s 35us/sample - loss: 0.7161 - val_loss: 0.9356
Epoch 2/100
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4480 - val_loss: 0.9624
Epoch 3/100
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4008 - val_loss: 3.9133
Epoch 4/100
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4058 - val_loss: 8.2736
Epoch 5/100
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4057 - val_loss: 11.7905
Epoch 6/100
11610/11610 [==============================] - 0s 26us/sample - loss: 0.4564 - val_loss: 1.0325
Epoch 7/100
11610/11610 [==============================] - 0s 26us/sample - loss: 0.3678 - val_loss: 0.4175
Epoch 8/100
11610/11610 [==============================] - 0s 26us/sample - loss: 0.3578 - val_loss: 0.3477
Epoch 9/100
11610/11610 [==============================] - 0s 26us/sample - loss: 0.34

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fd7509a9f28>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd7824d7828>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,...
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_trai